In [3]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'catboost'

In [1]:
import pandas

In [2]:
pandas.__version__

'1.1.5'

In [1]:
import sklearn
import catboost
import xgboost
import lightgbm

C:\Users\apurv\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
catboost.__version__

'0.26.1'

In [3]:
xgboost.__version__

'0.90'

In [4]:
lightgbm.__version__

'3.2.1'

In [2]:
cal_df = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv')
prices_df = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')
sales_df_wide = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_evaluation.csv')

NameError: name 'pd' is not defined

In [1]:
#selecting a random data point for prediction
input = sales_df_wide.sample()

NameError: name 'sales_df_wide' is not defined

In [ ]:
def final_fun_1(input):
    #there are many NaN values in event_name and event_type fields
    #Filling them with 'NoEvent' and 'None' resp
    cal_df['event_name_1'].fillna('NoEvent',inplace=True)
    cal_df['event_type_1'].fillna('None',inplace=True)
    cal_df['event_name_2'].fillna('NoEvent',inplace=True)
    cal_df['event_type_2'].fillna('None',inplace=True)
    #dropping redundant columns and changing datatypes to reduce memory usage
    cal_df.drop(['date','weekday'],axis=1,inplace=True)
    cal_df['wm_yr_wk'] = cal_df.wm_yr_wk.astype('int16')
    cal_df['d'] = cal_df.d.str[2:].astype('int16')
    for col in ['wday','month','snap_CA','snap_TX','snap_WI']:
        cal_df[col] = cal_df[col].astype('int8')
    for col in ['event_name_1','event_type_1','event_name_2','event_type_2']:
        cal_df[col] = cal_df[col].astype('category')
        
    # changing datatypes to reduce memory usage
    prices_df['store_id'] = prices_df.store_id.astype('category')
    prices_df['item_id'] = prices_df.item_id.astype('category')
    prices_df['wm_yr_wk'] = prices_df.wm_yr_wk.astype('int16')
    prices_df['sell_price'] = prices_df.sell_price.astype('float16')
    
    #converting the wide-form data frame to long-form so that columns from other 2 data frames can be merged
    train_df = pd.melt(sales_df_wide,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],\
                   var_name='d',value_name='units_sold')
    # changing datatypes to reduce memory usage
    train_df['d'] = train_df.d.str[2:].astype('int16')
    train_df['units_sold'] = train_df.units_sold.astype('int16')
    for col in ['id','item_id','dept_id','cat_id','store_id','state_id']:
        train_df[col] = train_df[col].astype('category') 

    #merging with 'cal_df'
    train_df = pd.merge(train_df,cal_df,on='d',how='left')
    #merging with 'prices_df'
    train_df = pd.merge(train_df,prices_df,on=['item_id','store_id','wm_yr_wk'],how='left')

    del cal_df,prices_df,sales_df_wide

    #sell_price data is not available for many rows. 
    #For previous weeks filling this data by mean sell_prices for the item_id and store_id pair
    train_df['sell_price'].fillna(train_df.groupby(['store_id','item_id'])['sell_price'].transform('mean'),inplace=True)

    #calculating lag features and filling NA values with 0
    for i in [7,14,21,28]:
        train_df['lag_'+str(i)] = train_df.groupby('id')['units_sold'].transform(lambda x:x.shift(i))
        train_df['lag_'+str(i)].fillna(0,inplace=True)

    #ref:https://www.kaggle.com/kyakovlev/m5-lags-features
    #rolling window mean features and filling NA values with 0
    for i in [7,14]:
        train_df['rolling_mean_'+str(i)] = train_df.groupby('id')['units_sold'].transform(lambda x:x.rolling(i).mean())
        train_df['rolling_mean_'+str(i)].fillna(0,inplace=True)
    #rolling window meadion features and filling NA values with 0
    for i in [7,14]:
        train_df['rolling_median_'+str(i)] = train_df.groupby('id')['units_sold'].transform(lambda x:x.rolling(i).median())
        train_df['rolling_median_'+str(i)].fillna(0,inplace=True)

    #dropping features 'wm_yr_wk' and 'year' as they are similar to 'd'
    train_df.drop(['wm_yr_wk','year'],axis=1,inplace=True)

    for i in ['lag_7','lag_14','lag_21','lag_28','rolling_mean_7','rolling_mean_14','rolling_median_7','rolling_median_14']:
        train_df[i] = train_df[i].astype('float16')
    
    y = train_df.loc[train_df.d.isin(range(500,1914))]['units_sold']
    y_test = train_df.loc[train_df.d.isin(range(1914,1942))]['units_sold']
    train_df.drop(['units_sold'],axis=1,inplace=True)
    x = train_df.loc[train_df.d.isin(range(500,1914))]
    x_test = train_df.loc[train_df.d.isin(range(1914,1942))]
    del train_df
    x_train1,x_train2,y_train1,y_train2 = train_test_split(x,y,test_size=0.5,random_state=42)
    del x,y

    #feature encoding
    le = LabelEncoder()
    x_train1_item_cat = le.fit_transform(x_train1.item_id.values)
    x_train2_item_cat = le.transform(x_train2.item_id.values)
    x_test_item_cat = le.transform(x_test.item_id.values)

    le_id = LabelEncoder()
    x_train1_id_cat = le_id.fit_transform(x_train1.id.values)
    x_train2_id_cat = le_id.transform(x_train2.id.values)
    x_test_id_cat = le_id.transform(x_test.id.values)

    x_train1_event_name_1 = le.fit_transform(x_train1.event_name_1.values)
    x_train2_event_name_1 = le.transform(x_train2.event_name_1.values)
    x_test_event_name_1 = le.transform(x_test.event_name_1.values)

    x_train1_event_type_1 = le.fit_transform(x_train1.event_type_1.values)
    x_train2_event_type_1 = le.transform(x_train2.event_type_1.values)
    x_test_event_type_1 = le.transform(x_test.event_type_1.values)

    x_train1_event_name_2 = le.fit_transform(x_train1.event_name_2.values)
    x_train2_event_name_2 = le.transform(x_train2.event_name_2.values)
    x_test_event_name_2 = le.transform(x_test.event_name_2.values)

    x_train1_event_type_2 = le.fit_transform(x_train1.event_type_2.values)
    x_train2_event_type_2 = le.transform(x_train2.event_type_2.values)
    x_test_event_type_2 = le.transform(x_test.event_type_2.values)

    x_train1['item_id'] = x_train1_item_cat
    x_train2['item_id'] = x_train2_item_cat
    x_test['item_id'] = x_test_item_cat

    x_train1['id'] = x_train1_id_cat
    x_train2['id'] = x_train2_id_cat
    x_test['id'] = x_test_id_cat

    x_train1['event_name_1'] = x_train1_event_name_1
    x_train2['event_name_1'] = x_train2_event_name_1
    x_test['event_name_1'] = x_test_event_name_1

    x_train1['event_type_1'] = x_train1_event_type_1
    x_train2['event_type_1'] = x_train2_event_type_1
    x_test['event_type_1'] = x_test_event_type_1

    x_train1['event_name_2'] = x_train1_event_name_2
    x_train2['event_name_2'] = x_train2_event_name_2
    x_test['event_name_2'] = x_test_event_name_2

    x_train1['event_type_2'] = x_train1_event_type_2
    x_train2['event_type_2'] = x_train2_event_type_2
    x_test['event_type_2'] = x_test_event_type_2

    del x_train1_item_cat,x_train1_id_cat,x_train1_event_name_1,x_train1_event_type_1,x_train1_event_name_2,x_train1_event_type_2
    del x_train2_item_cat,x_train2_id_cat,x_train2_event_name_1,x_train2_event_type_1,x_train2_event_name_2,x_train2_event_type_2
    del x_test_item_cat,x_test_id_cat,x_test_event_name_1,x_test_event_type_1,x_test_event_name_2,x_test_event_type_2

    x_train1_dept_cat = le.fit_transform(x_train1.dept_id.values)
    x_train2_dept_cat = le.transform(x_train2.dept_id.values)
    x_test_dept_cat = le.transform(x_test.dept_id.values)

    x_train1_cat = le.fit_transform(x_train1.cat_id.values)
    x_train2_cat = le.transform(x_train2.cat_id.values)
    x_test_cat = le.transform(x_test.cat_id.values)

    x_train1_store_cat = le.fit_transform(x_train1.store_id.values)
    x_train2_store_cat = le.transform(x_train2.store_id.values)
    x_test_store_cat = le.transform(x_test.store_id.values)

    x_train1_state_cat = le.fit_transform(x_train1.state_id.values)
    x_train2_state_cat = le.transform(x_train2.state_id.values)
    x_test_state_cat = le.transform(x_test.state_id.values)

    x_train1['dept_id'] = x_train1_dept_cat
    x_train2['dept_id'] = x_train2_dept_cat
    x_test['dept_id'] = x_test_dept_cat

    x_train1['cat_id'] = x_train1_cat
    x_train2['cat_id'] = x_train2_cat
    x_test['cat_id'] = x_test_cat

    x_train1['store_id'] = x_train1_store_cat
    x_train2['store_id'] = x_train2_store_cat
    x_test['store_id'] = x_test_store_cat

    x_train1['state_id'] = x_train1_state_cat
    x_train2['state_id'] = x_train2_state_cat
    x_test['state_id'] = x_test_state_cat

    del x_train1_dept_cat,x_train1_cat,x_train1_store_cat,x_train1_state_cat
    del x_train2_dept_cat,x_train2_cat,x_train2_store_cat,x_train2_state_cat
    del x_test_dept_cat,x_test_cat,x_test_store_cat,x_test_state_cat

    for i in ['item_id','event_name_1','event_type_1','event_name_2','event_type_2']:
        x_train1[i] = x_train1[i].astype('int16')
        x_train2[i] = x_train2[i].astype('int16')
        x_test[i] = x_test[i].astype('int16')
    
    for i in ['dept_id','cat_id','store_id','state_id']:
        x_train1[i] = x_train1[i].astype('int8')
        x_train2[i] = x_train2[i].astype('int8')
        x_test[i] = x_test[i].astype('int8')

In [34]:
#function to calculate assymmetric rmse,custom metric function
def armse(y_act,y_pred):
    score=0
    n = len(y_act)
    diff = np.array(y_pred) - np.array(y_act)
    for ele in diff:
        if ele<0:
            score += 4*(ele**2)
        else:
            score += ele**2
    return np.sqrt(score/n)

In [35]:
dt = DecisionTreeRegressor(max_depth=11,min_samples_split=3,random_state=0)
dt.fit(x_train1,y_train1)
y_pred2_dt = dt.predict(x_train2)
y_test_dt = dt.predict(x_test)

In [ ]:
rf = RandomForestRegressor(n_estimators=64,max_depth=12,min_samples_split=6,random_state=0,n_jobs=-1)
rf.fit(x_train1,y_train1)
y_pred2_rf = rf.predict(x_train2)
y_test_rf = rf.predict(x_test)

In [ ]:
xgb = XGBRegressor(learning_rate=0.140999404993131, max_depth=9, n_estimators=56,n_jobs=-1)
xgb.fit(x_train1,y_train1)
y_pred2_xgb = xgb.predict(x_train2)
y_test_xgb = xgb.predict(x_test)

In [ ]:
lgbm = LGBMRegressor(learning_rate=0.26661528358747144, max_depth=8, n_estimators=90, n_jobs=-1)
lgbm.fit(x_train1,y_train1)
y_pred2_lgbm = lgbm.predict(x_train2)
y_test_lgbm = lgbm.predict(x_test)

In [ ]:
cat = CatBoostRegressor(learning_rate=0.55,depth=3,logging_level="Silent")
cat.fit(x_train1,y_train1)
y_pred2_cat = cat.predict(x_train2)
y_test_cat = cat.predict(x_test)

In [ ]:
x_train_meta = np.vstack((y_pred2_dt,y_pred2_rf,y_pred2_xgb,y_pred2_lgbm,y_pred2_cat)).T
x_test_meta = np.vstack((y_test_dt,y_test_rf,y_test_xgb,y_test_lgbm,y_test_cat)).T

### **Linear Regression metamodel**

In [ ]:
lr = LinearRegression(n_jobs=-1)
lr.fit(x_train_meta,y_train2)
y_train_meta = lr.predict(x_train_meta)
y_test_meta = lr.predict(x_test_meta)

In [ ]:
d = {'id':le_id.inverse_transform(x_test['id'].values),'d':x_test['d'].values,'units':y_test_meta}
dt_cv = pd.DataFrame(data=d)
dt_cv = dt_cv.pivot_table(index='id',columns='d',values='units').reset_index()
names = {1914:'F1',1915:'F2',1916:'F3',1917:'F4',1918:'F5',1919:'F6',1920:'F7',1921:'F8',1922:'F9',1923:'F10',1924:'F11',1925:'F12',1926:'F13',1927:'F14',1928:'F15',1929:'F16',1930:'F17',1931:'F18',1932:'F19',1933:'F20',1934:'F21',1935:'F22',1936:'F23',1937:'F24',1938:'F25',1939:'F26',1940:'F27',1941:'F28'}
dt_cv.rename(columns=names,inplace=True)
dt_test = dt_cv.copy()
dt_cv['id'] = dt_cv.apply(lambda x: x['id'].replace('evaluation','validation'),axis=1)
dt_sub = pd.concat([dt_cv,dt_test],axis=0)
dt_sub.to_csv('customstackinglr.csv',index=False)
del dt_sub

In [ ]:
rmse_train = mse(y_train2,y_train_meta,squared=False)
armse_train = armse(y_train2,y_train_meta)
rmse_test = mse(y_test,y_test_meta,squared=False)
armse_test = armse(y_test,y_test_meta)

In [ ]:
print('Train: RMSE {} ARMSE {}'.format(rmse_train,armse_train))
print('Test: RMSE {} ARMSE {}'.format(rmse_test,armse_test))

In [ ]:
def final_fun_2():
    #there are many NaN values in event_name and event_type fields
    #Filling them with 'NoEvent' and 'None' resp
    cal_df['event_name_1'].fillna('NoEvent',inplace=True)
    cal_df['event_type_1'].fillna('None',inplace=True)
    cal_df['event_name_2'].fillna('NoEvent',inplace=True)
    cal_df['event_type_2'].fillna('None',inplace=True)
    #dropping redundant columns and changing datatypes to reduce memory usage
    cal_df.drop(['date','weekday'],axis=1,inplace=True)
    cal_df['wm_yr_wk'] = cal_df.wm_yr_wk.astype('int16')
    cal_df['d'] = cal_df.d.str[2:].astype('int16')
    for col in ['wday','month','snap_CA','snap_TX','snap_WI']:
        cal_df[col] = cal_df[col].astype('int8')
    for col in ['event_name_1','event_type_1','event_name_2','event_type_2']:
        cal_df[col] = cal_df[col].astype('category')
        
    # changing datatypes to reduce memory usage
    prices_df['store_id'] = prices_df.store_id.astype('category')
    prices_df['item_id'] = prices_df.item_id.astype('category')
    prices_df['wm_yr_wk'] = prices_df.wm_yr_wk.astype('int16')
    prices_df['sell_price'] = prices_df.sell_price.astype('float16')
    
    #converting the wide-form data frame to long-form so that columns from other 2 data frames can be merged
    train_df = pd.melt(sales_df_wide,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],\
                   var_name='d',value_name='units_sold')
    # changing datatypes to reduce memory usage
    train_df['d'] = train_df.d.str[2:].astype('int16')
    train_df['units_sold'] = train_df.units_sold.astype('int16')
    for col in ['id','item_id','dept_id','cat_id','store_id','state_id']:
        train_df[col] = train_df[col].astype('category') 

    #merging with 'cal_df'
    train_df = pd.merge(train_df,cal_df,on='d',how='left')
    #merging with 'prices_df'
    train_df = pd.merge(train_df,prices_df,on=['item_id','store_id','wm_yr_wk'],how='left')

    del cal_df,prices_df,sales_df_wide

    #sell_price data is not available for many rows. 
    #For previous weeks filling this data by mean sell_prices for the item_id and store_id pair
    train_df['sell_price'].fillna(train_df.groupby(['store_id','item_id'])['sell_price'].transform('mean'),inplace=True)

    #calculating lag features and filling NA values with 0
    for i in [7,14,21,28]:
        train_df['lag_'+str(i)] = train_df.groupby('id')['units_sold'].transform(lambda x:x.shift(i))
        train_df['lag_'+str(i)].fillna(0,inplace=True)

    #ref:https://www.kaggle.com/kyakovlev/m5-lags-features
    #rolling window mean features and filling NA values with 0
    for i in [7,14]:
        train_df['rolling_mean_'+str(i)] = train_df.groupby('id')['units_sold'].transform(lambda x:x.rolling(i).mean())
        train_df['rolling_mean_'+str(i)].fillna(0,inplace=True)
    #rolling window meadion features and filling NA values with 0
    for i in [7,14]:
        train_df['rolling_median_'+str(i)] = train_df.groupby('id')['units_sold'].transform(lambda x:x.rolling(i).median())
        train_df['rolling_median_'+str(i)].fillna(0,inplace=True)

    #dropping features 'wm_yr_wk' and 'year' as they are similar to 'd'
    train_df.drop(['wm_yr_wk','year'],axis=1,inplace=True)

    for i in ['lag_7','lag_14','lag_21','lag_28','rolling_mean_7','rolling_mean_14','rolling_median_7','rolling_median_14']:
        train_df[i] = train_df[i].astype('float16')
    
    y = train_df.loc[train_df.d.isin(range(500,1914))]['units_sold']
    y_test = train_df.loc[train_df.d.isin(range(1914,1942))]['units_sold']
    train_df.drop(['units_sold'],axis=1,inplace=True)
    x = train_df.loc[train_df.d.isin(range(500,1914))]
    x_test = train_df.loc[train_df.d.isin(range(1914,1942))]
    del train_df
    x_train1,x_train2,y_train1,y_train2 = train_test_split(x,y,test_size=0.5,random_state=42)
    del x,y

    #feature encoding
    le = LabelEncoder()
    x_train1_item_cat = le.fit_transform(x_train1.item_id.values)
    x_train2_item_cat = le.transform(x_train2.item_id.values)
    x_test_item_cat = le.transform(x_test.item_id.values)

    le_id = LabelEncoder()
    x_train1_id_cat = le_id.fit_transform(x_train1.id.values)
    x_train2_id_cat = le_id.transform(x_train2.id.values)
    x_test_id_cat = le_id.transform(x_test.id.values)

    x_train1_event_name_1 = le.fit_transform(x_train1.event_name_1.values)
    x_train2_event_name_1 = le.transform(x_train2.event_name_1.values)
    x_test_event_name_1 = le.transform(x_test.event_name_1.values)

    x_train1_event_type_1 = le.fit_transform(x_train1.event_type_1.values)
    x_train2_event_type_1 = le.transform(x_train2.event_type_1.values)
    x_test_event_type_1 = le.transform(x_test.event_type_1.values)

    x_train1_event_name_2 = le.fit_transform(x_train1.event_name_2.values)
    x_train2_event_name_2 = le.transform(x_train2.event_name_2.values)
    x_test_event_name_2 = le.transform(x_test.event_name_2.values)

    x_train1_event_type_2 = le.fit_transform(x_train1.event_type_2.values)
    x_train2_event_type_2 = le.transform(x_train2.event_type_2.values)
    x_test_event_type_2 = le.transform(x_test.event_type_2.values)

    x_train1['item_id'] = x_train1_item_cat
    x_train2['item_id'] = x_train2_item_cat
    x_test['item_id'] = x_test_item_cat

    x_train1['id'] = x_train1_id_cat
    x_train2['id'] = x_train2_id_cat
    x_test['id'] = x_test_id_cat

    x_train1['event_name_1'] = x_train1_event_name_1
    x_train2['event_name_1'] = x_train2_event_name_1
    x_test['event_name_1'] = x_test_event_name_1

    x_train1['event_type_1'] = x_train1_event_type_1
    x_train2['event_type_1'] = x_train2_event_type_1
    x_test['event_type_1'] = x_test_event_type_1

    x_train1['event_name_2'] = x_train1_event_name_2
    x_train2['event_name_2'] = x_train2_event_name_2
    x_test['event_name_2'] = x_test_event_name_2

    x_train1['event_type_2'] = x_train1_event_type_2
    x_train2['event_type_2'] = x_train2_event_type_2
    x_test['event_type_2'] = x_test_event_type_2

    del x_train1_item_cat,x_train1_id_cat,x_train1_event_name_1,x_train1_event_type_1,x_train1_event_name_2,x_train1_event_type_2
    del x_train2_item_cat,x_train2_id_cat,x_train2_event_name_1,x_train2_event_type_1,x_train2_event_name_2,x_train2_event_type_2
    del x_test_item_cat,x_test_id_cat,x_test_event_name_1,x_test_event_type_1,x_test_event_name_2,x_test_event_type_2

    x_train1_dept_cat = le.fit_transform(x_train1.dept_id.values)
    x_train2_dept_cat = le.transform(x_train2.dept_id.values)
    x_test_dept_cat = le.transform(x_test.dept_id.values)

    x_train1_cat = le.fit_transform(x_train1.cat_id.values)
    x_train2_cat = le.transform(x_train2.cat_id.values)
    x_test_cat = le.transform(x_test.cat_id.values)

    x_train1_store_cat = le.fit_transform(x_train1.store_id.values)
    x_train2_store_cat = le.transform(x_train2.store_id.values)
    x_test_store_cat = le.transform(x_test.store_id.values)

    x_train1_state_cat = le.fit_transform(x_train1.state_id.values)
    x_train2_state_cat = le.transform(x_train2.state_id.values)
    x_test_state_cat = le.transform(x_test.state_id.values)

    x_train1['dept_id'] = x_train1_dept_cat
    x_train2['dept_id'] = x_train2_dept_cat
    x_test['dept_id'] = x_test_dept_cat

    x_train1['cat_id'] = x_train1_cat
    x_train2['cat_id'] = x_train2_cat
    x_test['cat_id'] = x_test_cat

    x_train1['store_id'] = x_train1_store_cat
    x_train2['store_id'] = x_train2_store_cat
    x_test['store_id'] = x_test_store_cat

    x_train1['state_id'] = x_train1_state_cat
    x_train2['state_id'] = x_train2_state_cat
    x_test['state_id'] = x_test_state_cat

    del x_train1_dept_cat,x_train1_cat,x_train1_store_cat,x_train1_state_cat
    del x_train2_dept_cat,x_train2_cat,x_train2_store_cat,x_train2_state_cat
    del x_test_dept_cat,x_test_cat,x_test_store_cat,x_test_state_cat

    for i in ['item_id','event_name_1','event_type_1','event_name_2','event_type_2']:
        x_train1[i] = x_train1[i].astype('int16')
        x_train2[i] = x_train2[i].astype('int16')
        x_test[i] = x_test[i].astype('int16')
    
    for i in ['dept_id','cat_id','store_id','state_id']:
        x_train1[i] = x_train1[i].astype('int8')
        x_train2[i] = x_train2[i].astype('int8')
        x_test[i] = x_test[i].astype('int8')